In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
import glob
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.exc import IntegrityError

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize


In [47]:

#creo el entorno
engine = create_engine('sqlite:///deteccion_plagio_tp2.db')

#creo la base
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

# Actualiza las tablas en la base de datos
Base.metadata.create_all(engine)

#Defino las tablas

class Documentos(Base):
    __tablename__ = 'documentos'
    id = Column(Integer, primary_key=True)
    nombre_archivo = Column(String)
    texto = Column(String)
    
class Segmentos(Base):
    __tablename__ = 'segmentos'
    id_segmento = Column(Integer, primary_key=True)
    id_documento = Column(Integer, ForeignKey('documentos.id'))
    segmento_texto = Column(String)
    segmento_limpio = Column(String)
    init_s = Column(String)
    length_s = Column(Integer)
    
    documento = relationship("Documentos", backref="segmentos")

<ipython-input-47-43d5c16e5c9c>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [48]:
# Función para guardar el contenido de un archivo en la base de datos
def guardar_documento(ruta_archivo):
    nombre_archivo = os.path.basename(ruta_archivo)
    with open(ruta_archivo, 'r', encoding= "utf-8-sig") as archivo:
        contenido = archivo.read()

    nuevo_documento = Documentos(nombre_archivo=nombre_archivo, texto=contenido)
    session.add(nuevo_documento)

    try:
        session.commit()
        print(f"Documento '{nombre_archivo}' guardado correctamente.")
    except IntegrityError:
        session.rollback()
        print(f"Error al guardar el documento '{nombre_archivo}'.")



In [49]:
class Segmentation:
    
    def __init__(self, text):
        
        self.text = text
        #self.nombre = nombre
        
    def sentSegmentation(self):
        
        return sent_tokenize(self.text)
    
    def paraSegmentation(self):
        
        texto = self.text.split("\n\n")
        
        return list(filter(bool, texto))

In [50]:
def guardar_segmento(ruta_archivo, id_documento):
    
    nombre_archivo = os.path.basename(ruta_archivo)
    with open(ruta_archivo, 'r', encoding= "utf-8-sig") as archivo:
        text = archivo.read()
        
        Seg = Segmentation(text)

        sentSeg = Seg.sentSegmentation()

        paraSeg = Seg.paraSegmentation()
        
    for i, value in enumerate (sentSeg):

        nuevo_segmento = Segmentos(id_documento=id_documento, segmento_texto=value, init_s=i, length_s=int(len(value.strip())))
        session.add(nuevo_segmento)

    try:
        session.commit()
        print(f"Segmentos del archivo '{ruta_archivo}' guardados correctamente.")
    except IntegrityError:
        session.rollback()
        print(f"Error al guardar los segmentos del archivo '{ruta_archivo}'.")
        
        
        


In [51]:
def load_corpus_documents(carpeta):

    # Recorre los archivos de la carpeta
    for archivo in os.listdir(carpeta):
        ruta_archivo = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_archivo) and archivo.endswith('.txt'):
            guardar_documento(ruta_archivo)

In [52]:
def read_load_corpus_segments(carpeta):
    
    for archivo in os.listdir(carpeta):
        ruta_archivo = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_archivo) and archivo.endswith('.txt'):
            file = open(ruta_archivo, encoding= "utf-8-sig")
            
            print(archivo)

            documento = session.query(Documentos).filter_by(nombre_archivo=archivo).first()
            if documento:
                print(archivo, documento.id)
                guardar_segmento(ruta_archivo, documento.id)


In [53]:
def main():

    
    #Defino la carpeta donde esta el corpus y guardo los archivos en la base
    path = os.path.join(os.getcwd(), "corpus")
    load_corpus_documents(path)
    read_load_corpus_segments(path)

In [54]:
main()

Documento 'prueba_corpus1.txt' guardado correctamente.
Documento 'prueba_corpus2.txt' guardado correctamente.
prueba_corpus1.txt
prueba_corpus1.txt 3


OperationalError: (sqlite3.OperationalError) table segmentos has no column named length_s
[SQL: INSERT INTO segmentos (id_documento, segmento_texto, segmento_limpio, init_s, length_s) VALUES (?, ?, ?, ?, ?)]
[parameters: (3, 'adsvavfbfbagbgb\ngbga\nbag\nbgabagbagbgvca', None, 0, 39)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)